<a href="https://colab.research.google.com/github/LSDtopotools/lsdtt_notebooks/blob/master/basic_examples/chi_coordinate_from_basins.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Getting the chi coordinate from basins 

Last updated by Simon M Mudd on 24/05/2023

In this example I will download some data,select basins, and then extract the chi coordinate. 

In general I have a second window open to google maps. 
If you right click anywhere in google maps the latitude and longitude is copied to the clipboard, so you can then add these to the location data in the cells below. 

At the bottom of this notebook I also add an example of copying latitude and longitude entirely within a python notebook using folium and jinja.

If you use this software for your research, please cite https://doi.org/10.5281/zenodo.3245040

## If you are on colab

**If you are in the `docker_lsdtt_pytools` docker container, you do not need to do any of this. 
The following is for executing this code in the google colab environment only.**

If you are in the docker container you can skip to the **First get data** section. 

First we install `lsdviztools`. This will take around a minute. It is important you do this before the `condacolab` step. 

In [ ]:
!pip install lsdviztools &> /dev/null

Now we need to install lsdtopotools. We do this using something called `mamba`. To get `mamba` we install something called `condacolab`. 

In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install()

Now use mamba to install `lsdtopotools`. 
This step takes a bit over a minute. 

In [ ]:
!mamba install -y lsdtopotools &> /dev/null

The next line tests to see if it worked. If you get some output asking for a parameter file then `lsdtopotools` is installed. This notebook was tested on version 0.8.

In [ ]:
!lsdtt-basic-metrics -v

## Now get some data

We need to get some data to download. 

For this example we will work on the toe of the boot in Calabria, Italy. 

We are going to download data using the opentopography scraper that is included with `lsdviztools`. You will need to get an opentopography.org account and copy in your API key. 

You can sign up to an opentopography.org account here: https://portal.opentopography.org/myopentopo 

In [ ]:
import lsdviztools.lsdbasemaptools as bmt
from lsdviztools.lsdplottingtools import lsdmap_gdalio as gio

# YOU NEED TO PUT YOUR API KEY IN A FILE
your_OT_api_key_file = "my_OT_api_key.txt"

with open(your_OT_api_key_file, 'r') as file:
    print("I am reading you OT API key from the file "+your_OT_api_key_file)
    api_key = file.read().rstrip()
    print("Your api key starts with: "+api_key[0:4])

Dataset_prefix = "RC"
source_name = "COP30"

Xian_DEM = bmt.ot_scraper(source = source_name,
                        lower_left_coordinates = [37.88764759059215, 15.607351331344198], 
                        upper_right_coordinates = [38.29042048254062, 16.32690928209444],
                        prefix = Dataset_prefix, 
                        api_key_file = your_OT_api_key_file)
Xian_DEM.print_parameters()
Xian_DEM.download_pythonic()
DataDirectory = "./"
Fname = Dataset_prefix+"_"+source_name+".tif"
gio.convert4lsdtt(DataDirectory,Fname)

## Now get the basins

I add basins outlets into a pandas dataframe and then copy this to a file. 
The points below are obtained just by clicking in google maps and copying the resulting lat-long into the below code. 

In [ ]:
# Import pandas library
import pandas as pd

data = [ [37.94456243878353, 15.890659281175742],
         [37.93291942111813, 15.792125654538383],
         [38.06143176957989, 16.060604039819015],
         [38.094672987877786, 15.677112531901733],
         [38.20455853467961, 15.67642588617276],
         [38.12546843046537, 16.1230887795859]]

# Create the pandas DataFrame
df = pd.DataFrame(data, columns = ['latitude', 'longitude'])

df.to_csv("basin_outlets.csv",index=False)
df.head()

We can use the linux `cat` command to make sure the file is what we expect.

In [ ]:
!cat basin_outlets.csv

Now lets use *lsdtopotools* to get the basins. We first need to import the `lsdmapwrappers` module, and then run the code.

In [ ]:
import lsdviztools.lsdmapwrappers as lsdmw

## Get the basins
lsdtt_parameters = {"print_basin_raster" : "true",
                    "write_hillshade" : "true",
                    "get_basins_from_outlets" : "true",
                    "basin_outlet_csv" : "basin_outlets.csv",
                    "print_chi_data_maps" : "true"}
r_prefix = Dataset_prefix+"_"+source_name +"_UTM"
w_prefix = Dataset_prefix+"_"+source_name +"_UTM"
lsdtt_drive = lsdmw.lsdtt_driver(command_line_tool = "lsdtt-chi-mapping", 
                                 read_prefix = r_prefix,
                                 write_prefix= w_prefix,
                                 read_path = "./",
                                 write_path = "./",
                                 parameter_dictionary=lsdtt_parameters)
lsdtt_drive.print_parameters()
lsdtt_drive.run_lsdtt_command_line_tool()

Now we can print the map with an lsdviztools call.

In [ ]:
# uncomment this for debugging
#import lsdviztools.lsdmapwrappers as lsdmw
DataDirectory = "./"
Dataset_prefix = "RC"
source_name = "COP30"
r_prefix = Dataset_prefix+"_"+source_name +"_UTM"
Base_file = r_prefix

#%%capture
basins_img = lsdmw.PrintBasins_Complex(DataDirectory,Base_file,cmap="gist_earth", 
                             size_format="geomorphology",dpi=600, save_fig = True)

In [ ]:
print(basins_img)
from IPython.display import display, Image
display(Image(filename=basins_img, width=800))

## Looking at the chi data

With the call to `get_chi_data_maps` we have extracted some chi profiles. It is in a `csv` file with the string `chi_data_map` in the filename:

In [ ]:
!ls *chi_data_map*

Okay, lets see what is in this file, we load it in pandas and look at the columns:

In [ ]:
df = pd.read_csv("RC_COP30_UTM_chi_data_map.csv")
df.head()

### Chi data maps files

The chi data maps file has 8 columns:

* latitude, longitude: this should be obvious
* chi: the chi coordinate (in metres). This is drainage area integrated over flow distance (you can also calculate discharge over distance if you have a precipitation raster)
* elevation: elevation in metres or whatever vertical units your DEM is in
* flow_distance: this is the distance in metres from an outlet node
* drainage_area: the drainage3 area in m$^2$
* source_key: an integer value that denotes the headwater source of each pixel. Longer channels overwrite shorter channels. The lowest source in a basin is the longest channel. 
* basin_key: An integer that denotes the basin number. 

Lets make a simple plot of all the data using `folium`:

In [ ]:
# This is for the area threshold points
import folium

def plotDot(point):
    '''input: series that contains a numeric named latitude and a numeric named longitude
    this function creates a CircleMarker and adds it to your this_map'''
    folium.CircleMarker(location=[point.latitude, point.longitude],
                        radius=2,
                        weight=5).add_to(this_map)

In [ ]:
#create a map
this_map = folium.Map(prefer_canvas=True, tiles='Stamen Terrain')

#use df.apply(,axis=1) to "iterate" through every row in your dataframe
df.apply(plotDot, axis = 1)

#Set the zoom to the maximum possible
this_map.fit_bounds(this_map.get_bounds())

#Save the map to an HTML file
this_map.save('simple_dot_plot.html')

this_map

Okay, now lets isolate a single basin:

In [ ]:
df_b2 = df[(df['basin_key'] == 2)]

#create a map
this_map = folium.Map(prefer_canvas=True, tiles='Stamen Terrain')

#use df.apply(,axis=1) to "iterate" through every row in your dataframe
df_b2.apply(plotDot, axis = 1)

#Set the zoom to the maximum possible
this_map.fit_bounds(this_map.get_bounds())

this_map

### Plotting chi profiles

`lsdviztools` has some routines for plotting chi profiles, but you can just plot them with some scatter plots. Lets plot the chi profiles of a few basins:

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
basin = 4
df_single_basin = df[(df['basin_key'] == basin)]

fig = plt.figure()
ax = fig.add_subplot(1, 1,1)

plt.scatter(df_single_basin.chi,df_single_basin.elevation,c=df_single_basin.flow_distance)
plt.xlabel(r"$\chi$ (m)")
plt.ylabel("elevation (m)")
ax.text(0.1,0.9,"Basin "+str(basin),transform=ax.transAxes)
fig.show()

**Important note**

When chi is calculated using these routines, it is calculated from the entire DEM, and chi is based on the distance from all base level nodes, rather than the basin outlets. So in this case the base level nodes are all at the coast, which serevs as a common base level. 